# Generate a Modified Nested Set Index from NGA TDX-Hydro

This notebook demonstrates how to use functions in the [WikiWatershed/global-hydrography](https://github.com/WikiWatershed/global-hydrography) package to generate a modified nested set index using the TDX-Hydro datasets released by the [US National Geospatial-Intelligence Agency (NGA)](https://www.nga.mil).

This example notebook assumes that you have already downloaded the applicable data using the example provided in the `1_GetData.ipynb` notebook. This notebook also assumes that you will have completed the necessary setup steps outline in the **[Installation Instructions](README.md#get-started)** (and also completed as part of the notebook `1_GetData.ipynb`) 

# Python Imports

In this step we will import the necessary python dependencies for this example

In [1]:
from pathlib import Path
import re
from importlib import reload

import pyogrio
import geopandas as gpd

from global_hydrography.delineation.mnsi import modified_nest_set_index
from global_hydrography.preprocess import TDXPreprocessor

# Compile files that need to be processed

In this step we will compile a list of the files that need to be processed to have a modified nested set index. Note this step assumes that you have downloaded the files to the same directory and used the same naming convention as the `1_GetData.ipynb` example notebook. If you have opted to use a different location or naming convention you will need to modify this step accordingly.

In [2]:
# Confirm your current working directory (cwd) and repo/project directory
working_dir = Path.cwd()
project_dir = working_dir.parent
data_dir = project_dir / 'data_temp' # a temporary data directory that we .gitignore
tdx_dir = data_dir / 'nga'

In [3]:
#Scan the files in the data directory and only pull of the streamnet (blueline) files
files_to_process = []
for item in tdx_dir.iterdir():
    if item.is_file() and 'streamnet' in item.name:
        files_to_process.append(item)

In [4]:
files_to_process

[PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_1020011530_01.gpkg'),
 PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_7020038340_01.gpkg'),
 PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_7020038340_01.xedit.gpkg')]

# Explore Workflows and Test Functions

## Read Files

In [6]:
file = files_to_process[1]
file

PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_7020038340_01.gpkg')

In [7]:
#parse the file name to get the HDX Basin Id
tdx_basin_id = int(re.search("\d{10}",file.name).group(0))
tdx_basin_id

7020038340

In [8]:
info = pyogrio.read_info(file)
info

{'layer_name': 'TDX_streamnet_7020038340_01',
 'crs': 'EPSG:4326',
 'encoding': 'UTF-8',
 'fields': array(['LINKNO', 'DSLINKNO', 'USLINKNO1', 'USLINKNO2', 'DSNODEID',
        'strmOrder', 'Length', 'Magnitude', 'DSContArea', 'strmDrop',
        'Slope', 'StraightL', 'USContArea', 'WSNO', 'DOUTEND', 'DOUTSTART',
        'DOUTMID'], dtype=object),
 'dtypes': array(['int32', 'int32', 'int32', 'int32', 'int64', 'int32', 'float64',
        'int32', 'float64', 'float64', 'float64', 'float64', 'float64',
        'int32', 'float64', 'float64', 'float64'], dtype=object),
 'fid_column': 'fid',
 'geometry_name': 'geom',
 'geometry_type': 'LineString',
 'features': 140097,
 'total_bounds': (-89.8212222222222,
  24.5589999999989,
  -66.1413333333321,
  46.4454444444444),
 'driver': 'GPKG',
 'capabilities': {'random_read': True,
  'fast_set_next_by_index': True,
  'fast_spatial_filter': True,
  'fast_feature_count': True,
  'fast_total_bounds': True},
 'layer_metadata': {'DBF_DATE_LAST_UPDATE': '202

In [9]:
info['layer_metadata']

{'DBF_DATE_LAST_UPDATE': '2021-12-08'}

In [10]:
#open the file as GeoDataFrame, using the fastest direct method from issue #1
gdf = gpd.read_file(file, engine='pyogrio', layer=0, use_arrow=True)
gdf.info()
gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 140097 entries, 0 to 140096
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   LINKNO      140097 non-null  int32   
 1   DSLINKNO    140097 non-null  int32   
 2   USLINKNO1   140097 non-null  int32   
 3   USLINKNO2   140097 non-null  int32   
 4   DSNODEID    140097 non-null  int64   
 5   strmOrder   140097 non-null  int32   
 6   Length      140097 non-null  float64 
 7   Magnitude   140097 non-null  int32   
 8   DSContArea  140097 non-null  float64 
 9   strmDrop    140097 non-null  float64 
 10  Slope       140097 non-null  float64 
 11  StraightL   140097 non-null  float64 
 12  USContArea  140097 non-null  float64 
 13  WSNO        140097 non-null  int32   
 14  DOUTEND     140097 non-null  float64 
 15  DOUTSTART   140097 non-null  float64 
 16  DOUTMID     140097 non-null  float64 
 17  geometry    140097 non-null  geometry
dtypes: float64(9), g

,LINKNO,DSLINKNO,USLINKNO1,USLINKNO2,DSNODEID,strmOrder,Length,Magnitude,DSContArea,strmDrop,Slope,StraightL,USContArea,WSNO,DOUTEND,DOUTSTART,DOUTMID,geometry
0,0,1777,-1,-1,-1,1,3847.9,1,9567845.0,42.07,0.010933,3233.7,5254867.5,0,45853.6,49701.4,47777.5,"LINESTRING (-69.67822 46.41356, -69.67822 46.4..."
1,1,2369,-1,-1,-1,1,2251.3,1,8768556.0,34.66,0.015397,1749.2,4320561.0,1,44802.7,47054.1,45928.4,"LINESTRING (-69.68589 46.40778, -69.686 46.407..."
2,593,1777,-1,-1,-1,1,1469.3,1,8466694.0,11.98,0.008153,1286.2,4319318.0,593,45853.6,47322.9,46588.3,"LINESTRING (-69.67822 46.41356, -69.67811 46.4..."
3,1777,2369,0,593,-1,2,1050.9,2,19939082.0,0.91,0.000870,871.8,18034788.0,1777,44802.7,45853.6,45328.2,"LINESTRING (-69.68589 46.40778, -69.68589 46.4..."
4,2,4146,-1,-1,-1,1,3551.0,1,9120895.0,67.48,0.019002,2593.6,5267176.0,2,41041.1,44591.7,42816.4,"LINESTRING (-69.687 46.37911, -69.687 46.379, ..."


In [11]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Preprocess GeoDataFrame

In [ ]:
# Create an instance of the preprocessor class from our package
preprocessor = TDXPreprocessor()

In [12]:
test0_gdf = gdf.copy()

In [13]:
%%timeit
# apply preprocessing to make linkno globally unique
preprocessor.tdx_to_global_linkno(test0_gdf, tdx_basin_id)

7.57 ms ± 4.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
# apply preprocessing to make linkno globally unique
processed_gdf = preprocessor.tdx_to_global_linkno(gdf, tdx_basin_id)
processed_gdf.info()
processed_gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 140097 entries, 0 to 140096
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   LINKNO      140097 non-null  int32   
 1   DSLINKNO    140097 non-null  int32   
 2   USLINKNO1   140097 non-null  int32   
 3   USLINKNO2   140097 non-null  int32   
 4   DSNODEID    140097 non-null  int64   
 5   strmOrder   140097 non-null  int32   
 6   Length      140097 non-null  float64 
 7   Magnitude   140097 non-null  int32   
 8   DSContArea  140097 non-null  float64 
 9   strmDrop    140097 non-null  float64 
 10  Slope       140097 non-null  float64 
 11  StraightL   140097 non-null  float64 
 12  USContArea  140097 non-null  float64 
 13  WSNO        140097 non-null  int32   
 14  DOUTEND     140097 non-null  float64 
 15  DOUTSTART   140097 non-null  float64 
 16  DOUTMID     140097 non-null  float64 
 17  geometry    140097 non-null  geometry
dtypes: float64(9), g

,LINKNO,DSLINKNO,USLINKNO1,USLINKNO2,DSNODEID,strmOrder,Length,Magnitude,DSContArea,strmDrop,Slope,StraightL,USContArea,WSNO,DOUTEND,DOUTSTART,DOUTMID,geometry
0,750000000,750001777,-1,-1,-1,1,3847.9,1,9567845.0,42.07,0.010933,3233.7,5254867.5,0,45853.6,49701.4,47777.5,"LINESTRING (-69.67822 46.41356, -69.67822 46.4..."
1,750000001,750002369,-1,-1,-1,1,2251.3,1,8768556.0,34.66,0.015397,1749.2,4320561.0,1,44802.7,47054.1,45928.4,"LINESTRING (-69.68589 46.40778, -69.686 46.407..."
2,750000593,750001777,-1,-1,-1,1,1469.3,1,8466694.0,11.98,0.008153,1286.2,4319318.0,593,45853.6,47322.9,46588.3,"LINESTRING (-69.67822 46.41356, -69.67811 46.4..."
3,750001777,750002369,750000000,750000593,-1,2,1050.9,2,19939082.0,0.91,0.000870,871.8,18034788.0,1777,44802.7,45853.6,45328.2,"LINESTRING (-69.68589 46.40778, -69.68589 46.4..."
4,750000002,750004146,-1,-1,-1,1,3551.0,1,9120895.0,67.48,0.019002,2593.6,5267176.0,2,41041.1,44591.7,42816.4,"LINESTRING (-69.687 46.37911, -69.687 46.379, ..."


## Compute Modified Nested Set Index

In [15]:
test_gdf = processed_gdf.copy(deep=True)

In [16]:
%%timeit
# compute the modified nested set index is quite fast!! 
modified_nest_set_index(test_gdf)

9.72 s ± 160 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
#compute the modified nested set index
mnsi_gdf = modified_nest_set_index(processed_gdf)
mnsi_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 140097 entries, 0 to 140096
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   LINKNO         140097 non-null  int32   
 1   DSLINKNO       140097 non-null  int32   
 2   USLINKNO1      140097 non-null  int32   
 3   USLINKNO2      140097 non-null  int32   
 4   DSNODEID       140097 non-null  int64   
 5   strmOrder      140097 non-null  int32   
 6   Length         140097 non-null  float64 
 7   Magnitude      140097 non-null  int32   
 8   DSContArea     140097 non-null  float64 
 9   strmDrop       140097 non-null  float64 
 10  Slope          140097 non-null  float64 
 11  StraightL      140097 non-null  float64 
 12  USContArea     140097 non-null  float64 
 13  WSNO           140097 non-null  int32   
 14  DOUTEND        140097 non-null  float64 
 15  DOUTSTART      140097 non-null  float64 
 16  DOUTMID        140097 non-null  float64 
 17  ge

In [18]:
mnsi_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [19]:
mnsi_gdf.index

RangeIndex(start=0, stop=140097, step=1)

In [20]:
mnsi_gdf.columns

Index(['LINKNO', 'DSLINKNO', 'USLINKNO1', 'USLINKNO2', 'DSNODEID', 'strmOrder',
       'Length', 'Magnitude', 'DSContArea', 'strmDrop', 'Slope', 'StraightL',
       'USContArea', 'WSNO', 'DOUTEND', 'DOUTSTART', 'DOUTMID', 'geometry',
       'DISCOVER_TIME', 'FINISH_TIME', 'ROOT_ID'],
      dtype='object')

In [21]:
mnsi_gdf[['LINKNO', 'DSLINKNO', 'USLINKNO1', 'USLINKNO2', 'DSNODEID', 
          'DISCOVER_TIME', 'FINISH_TIME', 'ROOT_ID']]

,LINKNO,DSLINKNO,USLINKNO1,USLINKNO2,DSNODEID,DISCOVER_TIME,FINISH_TIME,ROOT_ID
0,750000000,750001777,-1,-1,-1,52,53,750021317
1,750000001,750002369,-1,-1,-1,49,50,750021317
2,750000593,750001777,-1,-1,-1,51,52,750021317
3,750001777,750002369,750000000,750000593,-1,50,53,750021317
4,750000002,750004146,-1,-1,-1,47,48,750021317
...,...,...,...,...,...,...,...,...
140092,750000587,-1,-1,-1,-1,1,2,750000587
140093,750001180,-1,-1,-1,-1,1,2,750001180
140094,750001772,-1,-1,-1,-1,1,2,750001772
140095,750000588,-1,-1,-1,-1,1,2,750000588


In [24]:
mnsi_gdf.ROOT_ID.value_counts().sort_values()

ROOT_ID
750005168        1
750000464        1
750075251        1
750000462        1
750070517        1
             ...  
750170058     4979
750301090     5929
750093395     6453
750127466     9771
750236632    14095
Name: count, Length: 1635, dtype: int64

In [21]:
# Drop columns with no value. See `sandbox/explore_data_sources.ipynb`
columns_to_drop = [
    'WSNO', # identical values to 'LINKNO'
    'DSNODEID', # all -1
]
mnsi_gdf.drop(columns=columns_to_drop, inplace=True)
mnsi_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 140097 entries, 0 to 140096
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   LINKNO         140097 non-null  int32   
 1   DSLINKNO       140097 non-null  int32   
 2   USLINKNO1      140097 non-null  int32   
 3   USLINKNO2      140097 non-null  int32   
 4   strmOrder      140097 non-null  int32   
 5   Length         140097 non-null  float64 
 6   Magnitude      140097 non-null  int32   
 7   DSContArea     140097 non-null  float64 
 8   strmDrop       140097 non-null  float64 
 9   Slope          140097 non-null  float64 
 10  StraightL      140097 non-null  float64 
 11  USContArea     140097 non-null  float64 
 12  DOUTEND        140097 non-null  float64 
 13  DOUTSTART      140097 non-null  float64 
 14  DOUTMID        140097 non-null  float64 
 15  geometry       140097 non-null  geometry
 16  DISCOVER_TIME  140097 non-null  int32   
 17  FI

## Write GeoParquet File

In [ ]:

#write back to the file
mnsi_gdf.to_parquet(
    
)

# Compute the modified nested set index

In this step we will loop through each of the files to be processed, open them as a GeoDataFrame, applied the modified nested set algorithm, and then write them back to the original file. Note this steps assumes you have used the same file naming convention as the `1_GetData.ipynb` example notebook. If your naming convention is different, you may need to modify the code below. 

In [25]:
# define a helper function for the operation
def compute_mnsi(file:Path, preprocessor:TDXPreprocessor) -> None:

    #parse the file name to get the HDX Basin Id
    tdx_basin_id = int(re.search("\d{10}",file.name).group(0))

    #open the file as GeoDataFrame
    gdf = pyogrio.read_dataframe(file, use_arrow=True)
    info = pyogrio.read_info(file)

    #apply preprocessing to make linkno globally unique
    gdf = preprocessor.tdx_to_global_linkno(gdf, tdx_basin_id)

    #compute the modified nested set index
    gdf = modified_nest_set_index(gdf)

    #write back to the file
    # FUNCTION BELOW OVER-WRITES FILE!!!
    # pyogrio.write_dataframe(gdf, file, layer=info['layer_name'], use_arrow=True)

In [26]:
#initialize a preprocessor instance
#we want to reuse this object to take advantage of the cached TDX Basin Id crosswalk
preprocessor = TDXPreprocessor()

file = files_to_process[1]

compute_mnsi(file, preprocessor)

/Users/aaufdenkampe/miniconda3/envs/hydrography/lib/python3.11/site-packages/pyogrio/raw.py:842: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
